In [1]:
%reset -f
%load_ext autoreload
%autoreload 2
# import sys
# sys.path.append('/home/saptarshi/Dhruv/Dissertation/ICC/')
from generate_attributions import *
from utils import CONFIG,evaluate_exp,PREFIX,aggregate_auc_or_sum

/home/saptarshi/anaconda3/envs/Dhruv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## German Credit Dataset

In [2]:
config = CONFIG['german']
mlp_path = PREFIX + 'models/german_mlp/german_mlp_seed_20/logs/version_2/checkpoints/epoch=69-step=910.ckpt'
scm_path = PREFIX + 'models/german_nf/german_nf_seed_20/logs/version_1/checkpoints/epoch=4-step=65.ckpt'
mlp_model = MLPLightning.load_from_checkpoint(mlp_path,model = MLP(config['num_features'],config['hidden_layers_mlp'],classification=config['classification']))
scm_model = CausalNF.load_from_checkpoint(scm_path,flow = flow(config['num_features'],config['gd_adjacency'],config['hidden_layers_flow']))

In [3]:
##generate attributions
ge_dict, time_dict, global_explanations = generate_global_exps(config,mlp_model.to('cpu'),scm_model.to('cpu'))

Max Iterations:  100
Starting Iteration:  1
Max Iterations:  100
Starting Iteration:  1
Max Iterations:  100
Starting Iteration:  1
Max Iterations:  100
Starting Iteration:  1
Max Iterations:  100
Starting Iteration:  1


In [4]:
# Load JSON
with open(PREFIX + "attributions/german/attribution.json", "r") as f:
    data = json.load(f)
global_explanations = data['global_explanations']    
generate_attr_plot(global_explanations,config)

In [5]:
ge_dict = data["ge_dict"]
evaluation_metrics,evaluation_metrics_per_sample = evaluate_exp(ge_dict,config,mlp_model.to('cpu'))
print(evaluation_metrics)


{'shap': {'pgi': {'k=1': [0.28330445289611816], 'k=2': [0.3505825400352478], 'k=3': [0.4646214246749878], 'k=4': [0.43991902470588684]}, 'pgu': {'k=1': [0.3754216432571411], 'k=2': [0.3217795193195343], 'k=3': [0.0998459905385971], 'k=4': [1.5497953143039922e-08]}}, 'lime': {'pgi': {'k=1': [0.28334569931030273], 'k=2': [0.3502984941005707], 'k=3': [0.33242443203926086], 'k=4': [0.4406302571296692]}, 'pgu': {'k=1': [0.37505313754081726], 'k=2': [0.3213956952095032], 'k=3': [0.36289605498313904], 'k=4': [1.5497953143039922e-08]}}, 'sp_lime': {'pgi': {'k=1': [0.09983356297016144], 'k=2': [0.21640270948410034], 'k=3': [0.3759300112724304], 'k=4': [0.44078755378723145]}, 'pgu': {'k=1': [0.46373602747917175], 'k=2': [0.43144139647483826], 'k=3': [0.2824808359146118], 'k=4': [1.5497953143039922e-08]}}, 'pfi': {'pgi': {'k=1': [0.28304028511047363], 'k=2': [0.4300941228866577], 'k=3': [0.4073118567466736], 'k=4': [0.440280556678772]}, 'pgu': {'k=1': [0.37520164251327515], 'k=2': [0.215272337198

In [6]:
auc_results,auc_show,sum_results,sum_show = aggregate_auc_or_sum(evaluation_metrics_per_sample,config)

# Print results nicely
for method, vals in auc_show.items():
    print(f"{method}: PGI AUC = {vals['pgi_auc']}, PGU AUC = {vals['pgu_auc']}")


shap: PGI AUC = 0.39227190220107633 +- 0.003540652656901855, PGU AUC = 0.20311211489140987 +- 0.0013519982998672383
lime: PGI AUC = 0.3482369937002659 +- 0.0028659059268991835, PGU AUC = 0.29060610953718424 +- 0.0030001524486968625
sp_lime: PGI AUC = 0.28754775900393725 +- 0.0020622096854759665, PGU AUC = 0.3152634181175381 +- 0.0027867276616399465
pfi: PGI AUC = 0.39968878393371904 +- 0.0032872440753583688, PGU AUC = 0.20401567661048225 +- 0.002363729275284718
ig: PGI AUC = 0.41846068248152735 +- 0.003855116180274921, PGU AUC = 0.16778183474205435 +- 0.0014410008553312871
sg: PGI AUC = 0.3243617471555869 +- 0.0024550322658796024, PGU AUC = 0.3205028007738292 +- 0.003611566061694538
itg: PGI AUC = 0.3244218061243494 +- 0.0024528432568287906, PGU AUC = 0.32002518477539227 +- 0.003631169078078209
icc_topo: PGI AUC = 0.4001388462384542 +- 0.003293728170951571, PGU AUC = 0.20404045618604869 +- 0.0023374039917381842
icc_shap: PGI AUC = 0.4000085904200872 +- 0.003301149364439707, PGU AUC = 0

In [7]:
for method, vals in sum_show.items():
    print(f"{method}: PGI SUM = {vals['pgi_sum']}, PGU SUM = {vals['pgu_sum']}")

shap: PGI SUM = 1.538427472114563 +- 0.013579691409443376, PGU SUM = 0.7970471978187561 +- 0.00560321962414369
lime: PGI SUM = 1.4066989421844482 +- 0.011600049720201832, PGU SUM = 1.0593448877334595 +- 0.010801035034488368
sp_lime: PGI SUM = 1.1329537630081177 +- 0.007471109471394204, PGU SUM = 1.1776583194732666 +- 0.010612462658552138
pfi: PGI SUM = 1.5607267618179321 +- 0.012703864092318806, PGU SUM = 0.7996478080749512 +- 0.008232324229185068
ig: PGI SUM = 1.6164050102233887 +- 0.01442082449454499, PGU SUM = 0.6910961270332336 +- 0.004963811365891736
sg: PGI SUM = 1.3348339796066284 +- 0.010256751686489894, PGU SUM = 1.1496567726135254 +- 0.012626634207107443
itg: PGI SUM = 1.3348983526229858 +- 0.010231023142816419, PGU SUM = 1.1480038166046143 +- 0.012716881146481396
icc_topo: PGI SUM = 1.5615147352218628 +- 0.012668638810004613, PGU SUM = 0.7996667623519897 +- 0.008202870957774442
icc_shap: PGI SUM = 1.561395287513733 +- 0.012738282262484016, PGU SUM = 0.8009902238845825 +- 0.0

## Lung Cancer Dataset

In [8]:
config = CONFIG['cancer']
mlp_path = PREFIX + 'models/cancer_mlp/cancer_mlp_seed_20/logs/version_1/checkpoints/epoch=62-step=2394.ckpt'
scm_path = PREFIX + 'models/cancer_nf/cancer_nf_seed_30/logs/version_1/checkpoints/epoch=143-step=5472.ckpt'
mlp_model = MLPLightning.load_from_checkpoint(mlp_path,model = MLP(config['num_features'],config['hidden_layers_mlp'],classification=config['classification']))
scm_model = CausalNF.load_from_checkpoint(scm_path,flow = flow(config['num_features'],config['gd_adjacency'],config['hidden_layers_flow']))

In [9]:
##generate attributions
ge_dict, time_dict, global_explanations = generate_global_exps(config,mlp_model.to('cpu'),scm_model.to('cpu'))

Max Iterations:  100
Starting Iteration:  1
Max Iterations:  100
Starting Iteration:  1
Max Iterations:  100
Starting Iteration:  1
Max Iterations:  100
Starting Iteration:  1
Max Iterations:  100
Starting Iteration:  1


In [10]:
# Load JSON
with open(PREFIX +"attributions/cancer/attribution.json", "r") as f:
    data = json.load(f)
global_explanations = data['global_explanations']    
generate_attr_plot(global_explanations,config)

In [11]:
ge_dict = data["ge_dict"]
evaluation_metrics,evaluation_metrics_per_sample = evaluate_exp(ge_dict,config,mlp_model.to('cpu'))
print(evaluation_metrics)


{'shap': {'pgi': {'k=1': [0.2134249359369278], 'k=2': [0.29935744404792786], 'k=3': [0.3327498733997345], 'k=4': [0.3791840672492981], 'k=5': [0.4066338837146759], 'k=6': [0.41710737347602844], 'k=7': [0.46727004647254944]}, 'pgu': {'k=1': [0.4375058114528656], 'k=2': [0.4146712124347687], 'k=3': [0.41035526990890503], 'k=4': [0.37464219331741333], 'k=5': [0.3473282754421234], 'k=6': [0.29170188307762146], 'k=7': [1.732276899701901e-08]}}, 'lime': {'pgi': {'k=1': [0.29096922278404236], 'k=2': [0.30258649587631226], 'k=3': [0.353669136762619], 'k=4': [0.3920283615589142], 'k=5': [0.419399619102478], 'k=6': [0.43815872073173523], 'k=7': [0.46740299463272095]}, 'pgu': {'k=1': [0.4173593521118164], 'k=2': [0.40230390429496765], 'k=3': [0.387849897146225], 'k=4': [0.3360205590724945], 'k=5': [0.2688675820827484], 'k=6': [0.21395662426948547], 'k=7': [1.732276899701901e-08]}}, 'sp_lime': {'pgi': {'k=1': [0.1240033432841301], 'k=2': [0.2449956238269806], 'k=3': [0.29447701573371887], 'k=4': [

In [12]:
auc_results,auc_show,sum_results,sum_show = aggregate_auc_or_sum(evaluation_metrics_per_sample,config)

# Print results nicely
for method, vals in auc_show.items():
    print(f"{method}: PGI AUC = {vals['pgi_auc']}, PGU AUC = {vals['pgu_auc']}")


shap: PGI AUC = 0.3625633541329039 +- 0.005434405297235846, PGU AUC = 0.3429086207996847 +- 0.003786609893473744
lime: PGI AUC = 0.3808380662197144 +- 0.004233050825645996, PGU AUC = 0.30294636474404896 +- 0.0048819548680504475
sp_lime: PGI AUC = 0.3480917841608688 +- 0.004916477859525521, PGU AUC = 0.3105389919873819 +- 0.0038085373972594114
pfi: PGI AUC = 0.41602669455111024 +- 0.005130484205305648, PGU AUC = 0.27678879071744933 +- 0.004258752647857307
ig: PGI AUC = 0.422581837479439 +- 0.005494635877927833, PGU AUC = 0.25079258081322126 +- 0.0032929148033632965
sg: PGI AUC = 0.3589843003017207 +- 0.005542195933280121, PGU AUC = 0.3302222847351287 +- 0.0037773637735847966
itg: PGI AUC = 0.34762859535351803 +- 0.0054549638024604325, PGU AUC = 0.34224078832828025 +- 0.003782746217289304
icc_topo: PGI AUC = 0.42503153905065527 +- 0.005425998826980188, PGU AUC = 0.2543862634777583 +- 0.0033561569825395553
icc_shap: PGI AUC = 0.42044343039807347 +- 0.005222422671933947, PGU AUC = 0.264047

In [13]:
for method, vals in sum_show.items():
    print(f"{method}: PGI SUM = {vals['pgi_sum']}, PGU SUM = {vals['pgu_sum']}")

shap: PGI SUM = 2.5157275199890137 +- 0.03761018707705641, PGU SUM = 2.2762045860290527 +- 0.02577621504553081
lime: PGI SUM = 2.6642143726348877 +- 0.029630891182910676, PGU SUM = 2.026357889175415 +- 0.032575742833868854
sp_lime: PGI SUM = 2.3839352130889893 +- 0.033556874192084724, PGU SUM = 2.0917434692382812 +- 0.025993595883807546
pfi: PGI SUM = 2.8757758140563965 +- 0.03499480850602114, PGU SUM = 1.8692442178726196 +- 0.028746006094831144
ig: PGI SUM = 2.9147613048553467 +- 0.03718131167278216, PGU SUM = 1.7137739658355713 +- 0.02292105178798616
sg: PGI SUM = 2.49440860748291 +- 0.03826398426950129, PGU SUM = 2.2000572681427 +- 0.02565349150021045
itg: PGI SUM = 2.426779270172119 +- 0.03775068623998722, PGU SUM = 2.2721328735351562 +- 0.025723693631361903
icc_topo: PGI SUM = 2.929060220718384 +- 0.036736378595597384, PGU SUM = 1.7351601123809814 +- 0.02327675880814974
icc_shap: PGI SUM = 2.902144193649292 +- 0.03555577828428349, PGU SUM = 1.7927380800247192 +- 0.0274519046325491

## Synthetic Data

In [14]:
config = CONFIG['syn']
mlp_path = PREFIX + 'models/syn_mlp/syn_mlp_seed_20/logs/version_3/checkpoints/epoch=12-step=416.ckpt'
scm_path = PREFIX + 'models/syn_nf/syn_nf_seed_20/logs/version_1/checkpoints/epoch=3-step=128.ckpt'
mlp_model = MLPLightning.load_from_checkpoint(mlp_path,model = MLP(config['num_features'],config['hidden_layers_mlp'],classification=config['classification']))
scm_model = CausalNF.load_from_checkpoint(scm_path,flow = flow(config['num_features'],config['gd_adjacency'],config['hidden_layers_flow']))

In [15]:
##generate attributions
ge_dict, time_dict, global_explanations = generate_global_exps(config,mlp_model.to('cpu'),scm_model.to('cpu'))

Max Iterations:  100
Starting Iteration:  1
Max Iterations:  100
Starting Iteration:  1
Max Iterations:  100
Starting Iteration:  1
Max Iterations:  100
Starting Iteration:  1
Max Iterations:  100
Starting Iteration:  1


In [16]:
# Load JSON
with open(PREFIX +"attributions/syn/attribution.json", "r") as f:
    data = json.load(f)
global_explanations = data['global_explanations']    
generate_attr_plot(global_explanations,config)

In [17]:
ge_dict = data["ge_dict"]
evaluation_metrics,evaluation_metrics_per_sample = evaluate_exp(ge_dict,config,mlp_model.to('cpu'))
print(evaluation_metrics)


{'shap': {'pgi': {'k=1': [0.029673971235752106], 'k=2': [0.04707258939743042], 'k=3': [0.06591856479644775]}, 'pgu': {'k=1': [0.04495923966169357], 'k=2': [0.024510297924280167], 'k=3': [1.8179417082819782e-08]}}, 'lime': {'pgi': {'k=1': [0.029624709859490395], 'k=2': [0.047130610793828964], 'k=3': [0.06602416932582855]}, 'pgu': {'k=1': [0.04484935477375984], 'k=2': [0.024553637951612473], 'k=3': [1.8179417082819782e-08]}}, 'sp_lime': {'pgi': {'k=1': [0.024534326046705246], 'k=2': [0.049360908567905426], 'k=3': [0.06593552976846695]}, 'pgu': {'k=1': [0.04707634076476097], 'k=2': [0.025295674800872803], 'k=3': [1.8179417082819782e-08]}}, 'pfi': {'pgi': {'k=1': [0.029754795134067535], 'k=2': [0.04940441623330116], 'k=3': [0.06606392562389374]}, 'pgu': {'k=1': [0.044876210391521454], 'k=2': [0.02522917464375496], 'k=3': [1.8179417082819782e-08]}}, 'ig': {'pgi': {'k=1': [0.029794491827487946], 'k=2': [0.047061506658792496], 'k=3': [0.06594599038362503]}, 'pgu': {'k=1': [0.04484673961997032

In [18]:
auc_results,auc_show,sum_results,sum_show = aggregate_auc_or_sum(evaluation_metrics_per_sample,config)

# Print results nicely
for method, vals in auc_show.items():
    print(f"{method}: PGI AUC = {vals['pgi_auc']}, PGU AUC = {vals['pgu_auc']}")


shap: PGI AUC = 0.04743443028690914 +- 0.00013928067187151286, PGU AUC = 0.023494963405731446 +- 0.0001143035625090633
lime: PGI AUC = 0.047477524693434435 +- 0.0001389063196778517, PGU AUC = 0.02348916217723551 +- 0.00011513451062266266
sp_lime: PGI AUC = 0.04729791758116335 +- 0.0001440316099778863, PGU AUC = 0.024416926212919254 +- 0.00010197038146697334
pfi: PGI AUC = 0.04865689010359347 +- 0.0001328717747295558, PGU AUC = 0.023833643932982037 +- 0.00010814747816156605
ig: PGI AUC = 0.04746587418330212 +- 0.00013850168800928604, PGU AUC = 0.023547237147577105 +- 0.0001138199178110556
sg: PGI AUC = 0.047490415507927534 +- 0.00013774286276957003, PGU AUC = 0.023515172407884772 +- 0.00011310415767934398
itg: PGI AUC = 0.04527036550454795 +- 0.00016699114319644885, PGU AUC = 0.02718037526666497 +- 7.710363740944899e-05
icc_topo: PGI AUC = 0.047501243685061735 +- 0.00014022401694885426, PGU AUC = 0.023440039810181286 +- 0.00011441043975770566
icc_shap: PGI AUC = 0.0474537610138456 +- 0.

In [19]:
for method, vals in sum_show.items():
    print(f"{method}: PGI SUM = {vals['pgi_sum']}, PGU SUM = {vals['pgu_sum']}")

shap: PGI SUM = 0.14266513288021088 +- 0.0004090698789623142, PGU SUM = 0.06946955621242523 +- 0.0003170963803254705
lime: PGI SUM = 0.14277948439121246 +- 0.00040584059613778137, PGU SUM = 0.0694030150771141 +- 0.0003183779760445929
sp_lime: PGI SUM = 0.13983075320720673 +- 0.0004403169752886709, PGU SUM = 0.07237202674150467 +- 0.00027899729517408966
pfi: PGI SUM = 0.14522314071655273 +- 0.00039258062871714525, PGU SUM = 0.0701054036617279 +- 0.00030408600231663236
ig: PGI SUM = 0.14280197024345398 +- 0.00040127236278756094, PGU SUM = 0.06951785832643509 +- 0.00031656121454502083
sg: PGI SUM = 0.1428375542163849 +- 0.0004011643068642873, PGU SUM = 0.06945367902517319 +- 0.00031337788020158446
itg: PGI SUM = 0.13612650334835052 +- 0.00048805773230604444, PGU SUM = 0.07903019338846207 +- 0.0002218846829196859
icc_topo: PGI SUM = 0.14286306500434875 +- 0.0004094366683683032, PGU SUM = 0.06928133964538574 +- 0.00031674708061853353
icc_shap: PGI SUM = 0.14271573722362518 +- 0.000406086060

## Test

In [2]:
config = CONFIG['syn']
mlp_path = '/home/dhruv/Files/Thesis/Dissertation/Code/models/syn_ann/syn_ann_seed_10/logs/version_0/checkpoints/epoch=161-step=5184.ckpt'
scm_path = '/home/dhruv/Files/Thesis/Dissertation/Code/models/syn_nf/syn_nf_seed_10/logs/version_0/checkpoints/epoch=19-step=640.ckpt'
mlp_model = MLPLightning.load_from_checkpoint(mlp_path,model = MLP(config['num_features'],config['hidden_layers_ann'],classification=False))
scm_model = CausalNF.load_from_checkpoint(scm_path,flow = flow(config['num_features'],config['gd_adjacency'],config['hidden_layers_flow']))
ge_dict, evaluation_metrics, time_dict, global_explanations = generate_global_exps(config,mlp_model,scm_model)

Max Iterations:  100
Starting Iteration:  1
Max Iterations:  100
Starting Iteration:  1
Max Iterations:  100
Starting Iteration:  1
Max Iterations:  100
Starting Iteration:  1
Max Iterations:  100
Starting Iteration:  1


In [3]:
ge_dict

{'ig': {'W': [0.008685602410414437],
  'Z': [0.5541948194138167],
  'X': [0.4371195781757688]},
 'itg': {'W': [0.008485306748260027],
  'Z': [0.497316907775009],
  'X': [0.49419778547673104]},
 'sg': {'W': [0.13136424456458626],
  'Z': [0.3860659244300918],
  'X': [0.48256983100532197]},
 'shap': {'W': [0.5803365821763931],
  'Z': [0.3543153621348831],
  'X': [0.06534805568872387]},
 'lime': {'W': [0.2837432524861816],
  'Z': [0.293367905107146],
  'X': [0.4228888424066725]},
 'sp_lime': {'X': [0.41602805243460733],
  'W': [0.2947343644233361],
  'Z': [0.2892375831420565]},
 'pfi': {'W': [0.2555289566516876],
  'Z': [0.2728649377822876],
  'X': [0.47160604596138]},
 'icc_topo': {'W': [0.07992755621671677],
  'Z': [0.6083893179893494],
  'X': [0.31013962626457214]},
 'icc_shap': {'W': [0.1287594437599182],
  'Z': [0.6630550622940063],
  'X': [0.20664207637310028]}}

In [9]:
generate_attr_plot(global_explanations,config)

In [7]:
# Load JSON
with open("/home/dhruv/Files/Thesis/Dissertation/Code/attributions/syn/attribution.json", "r") as f:
    data = json.load(f)

# Call the function
summary = compute_pgu_pgi_sums(data["evaluation_metrics"])

# Display the results
for method, scores in summary.items():
    print(f"{method}: PGU Sum = {scores['pgu_sum']:.6f}, PGI Sum = {scores['pgi_sum']:.6f}")


ig: PGU Sum = 0.128419, PGI Sum = 0.000436
itg: PGU Sum = 0.128419, PGI Sum = 0.000436
sg: PGU Sum = 0.128418, PGI Sum = 0.000437
shap: PGU Sum = 0.128420, PGI Sum = 0.000435
lime: PGU Sum = 0.128418, PGI Sum = 0.000437
sp_lime: PGU Sum = 0.128420, PGI Sum = 0.000435
pfi: PGU Sum = 0.128418, PGI Sum = 0.000437
icc_topo: PGU Sum = 0.128419, PGI Sum = 0.000436
icc_shap: PGU Sum = 0.128419, PGI Sum = 0.000436
